In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Code to Convert PDF to PNG

In [2]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [3]:
convert_pdf2img('Makro Invoices/Makro Invoices/Convert/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Convert/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI_page1.png']
###################################################################


['Makro Invoices/Makro Invoices/New/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI_page1.png']

In [275]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [5]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [460]:
Ima = "Makro Invoices/Makro Invoices/Converted/04.02.20_Carlos Santamaria Ruiz_213,64_page1.png"

In [461]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
tot = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
    
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
    
    
    
    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total factura" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a or "Importe:" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
            elif b[-1]=="€":
                dat["Total"] = b[-2]
                
            elif b[-1]=='€':
                dat["Total"] = b[-2]
            elif b[-1] == "|":
                dat["Total"] = b[-2]
            elif b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])


                
       
                
if dat["Total"] == 0 or dat["Total"]== "©" or (dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  or dat["Total"] == 'factura' or dat["Total"]=="FACTURA" or dat["Total"]== "EUROS" or dat["Total"] == "©" or dat["Total"]=="TOTAL":
        tot = []
        im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
        tex=(pytesseract.image_to_string(img))
        text_list = tex.split("\n")
        text_list = text_list
        print(text_list)
        for idx, a in enumerate(text_list):
            if(a.strip() != ""):
                if re.search('[0-9]{2},[0-9]{2}',a):
                    b = re.findall('[0-9]{2},[0-9]{2}',a)
                    print(b,'this is total')
                    dat['Total'] = b
                
                
                
                
        
            
            
        

<class 'list'>
Factura 1 000155 1 04/02/2020
['Factura', '1', '000155', '1', '04/02/2020']
['Factura', '1', '000155', '1', '04/02/2020']
Case Total
['RTICULO', 'DESCRIPCION', 'PESO', 'PRECIO', 'KG', 'SUBTOTAL']
SUBTOTAL
Wall time: 6.7 s


In [462]:
dat

{'Vat Number': '',
 'Total': 'SUBTOTAL',
 'Date': '04/02/2020',
 'Invoice_Number': ''}

In [415]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number Total        Date Invoice_Number
0  B76761915     ©  03/02/2020               


In [416]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data['Vat Number']= data['Vat Number'].str.replace('/', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until



,Vat Number,Total,Date,Invoice_Number
0,B76761915,©,03/02/2020,


In [335]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

NameError: name 'formats' is not defined

In [240]:
df2 = formats.query("last == True")

In [241]:
df2

,formato,nombre,nif,last
3158,\d\d\d\d\d\d\d\d\d\d,EMBUTIDOS DE TENERIFE S.A.,A38003414,True


In [242]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile(inv)
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [243]:
dat

{'Vat Number': ['A-38003414'],
 'Total': '154,05',
 'Date': '27-05-2021',
 'Invoice_Number': '1400804213'}

In [463]:
im = Image.open(Ima)

In [464]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [465]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [466]:
if dataframes[0].shape[1] >=4 :
    this_data = pd.DataFrame(dataframes[0])

elif  dataframes[1].shape[1] >=4 :
    this_data = pd.DataFrame(dataframes[1])
  
elif dataframes[2].shape[1] >=4:
    this_data = pd.DataFrame(dataframes[2])

else:
    this_data = pd.DataFrame(dataframes[3])
    
this_data

,ARTÍCULO,DESCRIPCIÓN,,PESO,PRECIO KG,SUBTOTAL,DTO.,TOTAL
0,580,SOLOMILLO DE CERDO SECO,373870,"10,000","5,35","53,50",,"53,50"
1,74255,CHOCOS LIMPIOS 2/4 (,348423,"10,000","7,00","70,00",,"70,00"
2,1000041.,CODILLO DE CERDO C/H,107651,"24,250","2,95","71,54",,"71,54"
3,1080,MUSLO DE POLLO C/H C/C GRANEL,361514,"10,000","1,65","16,50",,"16,50"
4,,that,,,,to,,


In [306]:
dat

{'Vat Number': ['A-38003414'],
 'Total': '90,21',
 'Date': '31/05/2020',
 'Invoice_Number': 'OPERACIONESTA'}

In [249]:
json_f = {}
json_f['file'] = Ima
json_f['labels'] = dat
json_f['tables'] = this_data
print(json_f)

{'file': 'Makro Invoices/Makro Invoices/Converted/guimar040621_page_001_page-0001.jpg', 'labels': {'Vat Number': ['A-38003414'], 'Total': '154,05', 'Date': '27-05-2021', 'Invoice_Number': '1400804213'}, 'tables':     REF.                    DESCRIPCIÓN    Lote Bultos/Unids  Cantidad  \
0    376  PREPARADO CÁRNICO PEPITOS VAC  493003         3 CJ  18,00 KG   
1  71239  BURGER MEAT VACUNO SUPRA 150G  510220         1 CJ   4,50 KG   
2  76100  LOMOS DE ATUN GRANEL 25KG CON  446247         1 UN   3,50 KG   

       Precio Descuento   Neto   Total IGIC       
0  6,870 €/KG       10%  6,183  111,29       (1)  
1  6,140 €/KG       10%  5,526   24,87   3%       
2  5,440 €/KG       10%  4,896   17,14            }


In [250]:
dat

{'Vat Number': ['A-38003414'],
 'Total': '154,05',
 'Date': '27-05-2021',
 'Invoice_Number': '1400804213'}

In [264]:
this_data.head()

,Pos.,Cant.,Denominación /Referencia,Precio Unit.,Precio Total EUR
0,00100,"1,0",Cuota de Arrendamiento según contrato de refer...,"44,69","44,69"
1,,,"Importe Neto IGIC 7,000 %","44,69","44,69 3,12"
2,,,Total Factura EUR,,"47,81"
3,,,Condiciones de pago : VENCIMIENTO INMEDIATO,,
4,,,PAGADO,,


In [467]:
name = ''
index = 0
for i in this_data.columns:
    if (i == 'DESCRIPCIÓN') or (i=="Artículo/ Cod.cliente")or (i == 'DESCRIPCIÓN:')  or(i =="CONCEPTO")or (i=="UDS") or (i =="Concepto") or (i=="Descripción")  or (i=='DESCRIPCION') or (i=="Description articulo") or (i=="Description") or (i == "DESCRIPCIÓN ARTICULO") or (i=="Unidades Descripción")or (i=="PRODUCTO") or (i=="Articulo") or (i=='Denominación /Referencia'):
        print(i)
        match = index
        name = i
    index+=1

DESCRIPCIÓN


In [468]:
this = this_data[name]

In [469]:
dataf  =  pd.DataFrame(this,columns=[name])

In [470]:
dataf

,DESCRIPCIÓN
0,SOLOMILLO DE CERDO SECO
1,CHOCOS LIMPIOS 2/4 (
2,CODILLO DE CERDO C/H
3,MUSLO DE POLLO C/H C/C GRANEL
4,that


In [471]:
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        price =i

PRECIO KG
Im condition 1


In [472]:
price_col = this_data[price]
price_data = pd.DataFrame(price_col,columns=[price])
price_data

,PRECIO KG
0,"5,35"
1,"7,00"
2,"2,95"
3,"1,65"
4,


In [473]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    

DESCRIPCIÓN
Im condition 4
DTO.
Im condition 1


In [474]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

Im condition 6
PESO


In [475]:
Cantidad = this_data[cat]
Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
Cantidad_data

,PESO
0,"10,000"
1,"10,000"
2,"24,250"
3,"10,000"
4,


In [476]:
discount = this_data[disc]
discount_data = pd.DataFrame(discount,columns=[disc])
discount_data

,DTO.
0,
1,
2,
3,
4,


In [477]:
#df = pd.DataFrame([dataf,price_data], columns=['Description','Price'])
df2 = pd.concat([dataf,price_data,Cantidad_data,discount_data],axis=1, join='inner')


DCTO

In [478]:
df2

,DESCRIPCIÓN,PRECIO KG,PESO,DTO.
0,SOLOMILLO DE CERDO SECO,"5,35","10,000",
1,CHOCOS LIMPIOS 2/4 (,"7,00","10,000",
2,CODILLO DE CERDO C/H,"2,95","24,250",
3,MUSLO DE POLLO C/H C/C GRANEL,"1,65","10,000",
4,that,,,
